### <left> High-Dimensional Deep Learning - 5A
## <center>*Mini-project n° 3 – SSL for Anomaly Detection*</center>
### <center>2024-2025</center>
<div style="text-align: right;">
    <b>Project 3/4</b><br>
<div style="text-align: left;">
<b>Noms :</b> Maïmouna Gadji, Phuc-Luan Nguyen, Maddie Perez et Julia Soufflet<br>

### Objectif : 

L'objectif de ce projet est de 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
# Training images
pathb="./MVTec_AD/bottle"
pathh = "./MVTec_AD/hazelnut"
pathc = "./MVTec_AD/capsule"
patht = "./MVTec_AD/toothbrush"

# I - Partie 1 : Train SSL
## Pretext tasks 1 : masked autoencoder

### Sur MVTec_AD

### Sur Auto_VI

## Pretext tasks 2 : inpainting model

### Sur MVTec_AD

### Sur Auto_VI

## Pretext tasks 3 : colorizing model

### Sur MVTec_AD

### Sur Auto_VI

# II - Partie 2 :
Use the model loss as an anomaly score and evaluate the anomaly score’s
discriminative power by plotting the ROC curve and the AUROC metric.